# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = "..\WeatherPy\my_data.csv"
cities_df = pd.read_csv(cities)
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.6000,73.0833,85.15,68,100,14.61,MV,1651351809
1,Waddān,29.1614,16.1390,81.12,16,73,24.61,LY,1651351809
2,Bluff,-46.6000,168.3333,63.81,64,100,11.83,NZ,1651351809
3,Dikson,73.5069,80.5464,17.49,94,100,19.01,RU,1651351809
4,New Norfolk,-42.7826,147.0587,58.50,67,100,4.00,AU,1651351809


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)

heat_layer =gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False,max_intensity=100,
                                point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
perfect_df = cities_df.loc[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70) \
                                    & (cities_df["Wind Speed"] < 10) \
                                    & (cities_df["Cloudiness"] == 0)].dropna()
perfect_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,Gerāsh,27.6652,54.1371,71.74,47,0,5.75,IR,1651351809
238,Quelimane,-17.8786,36.8883,75.15,88,0,3.44,MZ,1651351809
310,Ondjiva,-17.0667,15.7333,70.32,41,0,5.53,AO,1651351809
351,Santiago,-33.4569,-70.6483,74.28,32,0,9.22,CL,1651351809
353,Mahajanga,-15.7167,46.3167,78.17,79,0,8.70,MG,1651351809
421,Dābhol,17.6000,73.1667,77.79,83,0,3.47,IN,1651351809
432,Havelock,34.8791,-76.9013,75.20,64,0,3.44,US,1651351809


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = perfect_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
37,Gerāsh,IR,71.74,27.6652,54.1371,
238,Quelimane,MZ,75.15,-17.8786,36.8883,
310,Ondjiva,AO,70.32,-17.0667,15.7333,
351,Santiago,CL,74.28,-33.4569,-70.6483,
353,Mahajanga,MG,78.17,-15.7167,46.3167,
421,Dābhol,IN,77.79,17.6000,73.1667,
432,Havelock,US,75.20,34.8791,-76.9013,


In [6]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
37,Gerāsh,IR,71.74,27.6652,54.1371,Sa'adat House Gerash
238,Quelimane,MZ,75.15,-17.8786,36.8883,Hotel Flamingo
310,Ondjiva,AO,70.32,-17.0667,15.7333,COMBUESA
351,Santiago,CL,74.28,-33.4569,-70.6483,Hotel Panamericano
353,Mahajanga,MG,78.17,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
421,Dābhol,IN,77.79,17.6000,73.1667,Ashwamedh Resort
432,Havelock,US,75.20,34.8791,-76.9013,Sherwood Motel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

lodging = hotel_df[["Lat", "Lng"]]
temp = hotel_df["Max Temp"].astype(float)

heat_layer =gmaps.heatmap_layer(lodging, weights=temp,
                                dissipating=False,max_intensity=10,
                                point_radius=3)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))